In [1]:
# set SLM

from meadowlark import Meadowlark

slm = Meadowlark(
    verbose=True,
    sdk_path="C:\\Program Files\\Meadowlark Optics\\Blink OverDrive Plus",
    # lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\slm5691_at635.LUT",
    lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\1920x1152_linearVoltage.LUT",
    wav_um=0.550,
    pitch_um=(9.2, 9.2),
)

print(slm.shape)


Validating DPI awareness...success
Loading Blink SDK libraries...success
Initializing SDK...success
Found 1 SLM controller(s)
Loading LUT file...success
(1152, 1920)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Constants
WAVELENGTH = 550e-9 
PIXEL_SIZE = 9.2e-6  # 9.2 μm in meters
DEFAULT_FL_MM = 241
DEFAULT_2PI_VALUE = 190  # Default SLM value for 2π phase

# Function to calculate linear phase for beam steering
def calculate_linear_phase(shape, angle_x_mrad, angle_y_mrad, wavelength=WAVELENGTH, pixel_size=PIXEL_SIZE):
    """
    Calculate linear phase gradient for beam steering.
    
    Parameters:
    -----------
    shape : tuple
        The shape of the SLM (height, width)
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    wavelength : float
        Wavelength of light in meters
    pixel_size : float
        Size of the SLM pixel in meters
    
    Returns:
    --------
    linear_phase : ndarray
        The linear phase pattern (0 to 2π)
    """
    height, width = shape
    
    # Convert milliradians to radians
    angle_x_rad = angle_x_mrad * 1e-3
    angle_y_rad = angle_y_mrad * 1e-3
    
    # Calculate phase gradients (rad/m)
    # For small angles: sin(θ) ≈ θ
    # Phase gradient = 2π * sin(θ) / λ
    phase_gradient_x = 2 * np.pi * np.sin(angle_x_rad) / wavelength
    phase_gradient_y = 2 * np.pi * np.sin(angle_y_rad) / wavelength
    
    # Create coordinate grids in meters
    y, x = np.indices(shape)
    x_meters = (x - width/2) * pixel_size
    y_meters = (y - height/2) * pixel_size
    
    # Calculate linear phase
    linear_phase = phase_gradient_x * x_meters + phase_gradient_y * y_meters
    
    return linear_phase

# Function to calculate Fresnel lens phase pattern using exact formula for a microlens array within ROI
def calculate_fresnel_microlens_array(shape, focal_length, rows, cols, roi_center_x, roi_center_y, 
                                      roi_width, roi_height, angle_x_mrad, angle_y_mrad,
                                      wavelength=WAVELENGTH, pixel_size=PIXEL_SIZE):
    """
    Calculate the phase pattern for a Fresnel microlens array within a specified ROI with beam steering.
    
    Parameters:
    -----------
    shape : tuple
        The shape of the SLM (height, width)
    focal_length : float
        Focal length of the lenses in mm (positive for converging, negative for diverging)
    rows : int
        Number of microlens rows in the array
    cols : int
        Number of microlens columns in the array
    roi_center_x, roi_center_y : int
        Center position of the ROI in pixels
    roi_width, roi_height : int
        Width and height of the ROI in pixels
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    wavelength : float
        Wavelength of light in meters
    pixel_size : float
        Size of the SLM pixel in meters
    
    Returns:
    --------
    phase : ndarray
        The phase pattern (0 to 2π)
    roi_mask : ndarray
        Boolean mask for the ROI
    """
    height, width = shape
    
    # Convert focal length to meters
    focal_length_m = focal_length * 1e-3
    
    # Calculate ROI boundaries
    roi_left = max(0, int(roi_center_x - roi_width // 2))
    roi_right = min(width, int(roi_center_x + roi_width // 2))
    roi_top = max(0, int(roi_center_y - roi_height // 2))
    roi_bottom = min(height, int(roi_center_y + roi_height // 2))
    
    # Actual ROI dimensions (may be smaller than requested if near edges)
    actual_roi_width = roi_right - roi_left
    actual_roi_height = roi_bottom - roi_top
    
    # Calculate the size of each microlens (in pixels)
    lens_height = actual_roi_height // rows
    lens_width = actual_roi_width // cols
    
    # Create coordinate grid for the full SLM
    y, x = np.indices(shape)
    
    # Initialize the phase array
    phase = np.zeros(shape)
    
    # Create ROI mask
    roi_mask = (x >= roi_left) & (x < roi_right) & (y >= roi_top) & (y < roi_bottom)
    
    # Calculate the linear phase for beam steering
    linear_phase = calculate_linear_phase(shape, angle_x_mrad, angle_y_mrad, wavelength, pixel_size)
    
    # Calculate the center offsets for each lens in the array
    for r in range(rows):
        for c in range(cols):
            # Calculate the center of this microlens (in pixels) relative to ROI
            center_y = roi_top + r * lens_height + lens_height // 2
            center_x = roi_left + c * lens_width + lens_width // 2
            
            # Calculate the region this microlens covers
            y_start = roi_top + r * lens_height
            y_end = roi_top + (r + 1) * lens_height
            x_start = roi_left + c * lens_width
            x_end = roi_left + (c + 1) * lens_width
            
            # Handle edge cases where division might not be exact
            y_end = min(y_end, roi_bottom)
            x_end = min(x_end, roi_right)
            
            # Get the region for this microlens
            region_y = slice(y_start, y_end)
            region_x = slice(x_start, x_end)
            
            # Calculate distances from center in meters
            x_dist = (x[region_y, region_x] - center_x) * pixel_size
            y_dist = (y[region_y, region_x] - center_y) * pixel_size
            r_squared = x_dist**2 + y_dist**2
            
            # Calculate phase using the exact formula:
            # φ(x,y) = (2π/λ) * (f - sqrt(f² + x² + y²))
            f_squared = focal_length_m**2
            phase_calc = (2 * np.pi / wavelength) * (focal_length_m - np.sqrt(f_squared + r_squared))
            
            # Add the linear phase for this region
            phase_calc += linear_phase[region_y, region_x]
            
            # Assign the calculated phase to this region
            phase[region_y, region_x] = phase_calc
    
    # Wrap phase to [0, 2π) range
    phase = phase % (2 * np.pi)
    
    return phase, roi_mask, (lens_width, lens_height)

# Function to calculate Airy disk diameter
def calculate_airy_disk(focal_length, aperture_width, aperture_height, wavelength=WAVELENGTH):
    """
    Calculate the diffraction limited spot size (Airy disk diameter)
    
    Parameters:
    -----------
    focal_length : float
        Focal length in meters
    aperture_width, aperture_height : float
        Width and height of the aperture in meters
    wavelength : float
        Wavelength of light in meters
    
    Returns:
    --------
    airy_disk_diameter : float
        Estimated Airy disk diameter in micrometers
    """
    # For square apertures, use the average dimension for the calculation
    aperture_size = (aperture_width + aperture_height) / 2
    
    # Calculate f-number
    f_number = focal_length / aperture_size
    
    # Calculate Airy disk diameter
    airy_disk_diameter = 2.44 * wavelength * f_number
    
    # Convert to micrometers for easier reading
    return airy_disk_diameter * 1e6

# Create focal length adjustment buttons
def update_phase(focal_length_coarse, focal_length_fine, rows, cols, 
                 roi_center_x, roi_center_y, roi_width, roi_height, is_negative,
                 angle_x_mrad, angle_y_mrad, two_pi_value):
    """
    Update the phase pattern based on the current parameters
    
    Parameters:
    -----------
    focal_length_coarse : float
        Coarse adjustment of focal length in mm (integer steps)
    focal_length_fine : float
        Fine adjustment of focal length in mm (decimal steps)
    rows : int
        Number of microlens rows
    cols : int
        Number of microlens columns
    roi_center_x, roi_center_y : int
        Center position of the ROI in pixels
    roi_width, roi_height : int
        Width and height of the ROI in pixels
    is_negative : bool
        True if the lens should be negative (diverging)
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    two_pi_value : int
        SLM value corresponding to 2π phase
    
    Returns:
    --------
    phi : ndarray
        The calculated phase pattern as uint8
    """
    # Get SLM shape
    shape = slm.shape
    
    # Combine coarse and fine focal length adjustments
    focal_length = focal_length_coarse + focal_length_fine
    
    # Apply the sign to the focal length
    actual_focal_length = -focal_length if is_negative else focal_length
    
    # Calculate Fresnel phase for microlens array with beam steering
    phase, roi_mask, (lens_width, lens_height) = calculate_fresnel_microlens_array(
        shape, 
        actual_focal_length, 
        rows, 
        cols,
        roi_center_x,
        roi_center_y,
        roi_width,
        roi_height,
        angle_x_mrad,
        angle_y_mrad
    )
    
    # Create checkerboard for regions outside ROI
    checkerboard = create_checkerboard(shape)
    
    # Combine the patterns - lens array inside ROI, checkerboard outside
    combined_phase = np.where(roi_mask, phase, checkerboard)
    
    # Convert to uint8 using the calibrated 2π value
    phi = np.uint8(combined_phase / (2 * np.pi) * two_pi_value)
    
    # Upload to SLM
    slm.set_phase(phi)
    
    # Display visualization
    plt.figure(figsize=(12, 6))
    plt.imshow(phi, cmap='gray')
    plt.colorbar(label=f'Phase (0-{two_pi_value})')
    
    # Draw ROI boundary
    roi_left = roi_center_x - roi_width // 2
    roi_right = roi_center_x + roi_width // 2
    roi_top = roi_center_y - roi_height // 2
    roi_bottom = roi_center_y + roi_height // 2
    
    plt.plot([roi_left, roi_right, roi_right, roi_left, roi_left], 
             [roi_top, roi_top, roi_bottom, roi_bottom, roi_top], 
             'r-', linewidth=2, label='ROI')
    
    # Calculate Airy disk diameter
    lens_width_m = lens_width * PIXEL_SIZE
    lens_height_m = lens_height * PIXEL_SIZE
    airy_disk = calculate_airy_disk(abs(focal_length) * 1e-3, lens_width_m, lens_height_m)
    
    # Calculate deflection angles in degrees for display
    angle_x_deg = angle_x_mrad * 0.0573  # Convert mrad to degrees
    angle_y_deg = angle_y_mrad * 0.0573
    
    plt.title(f'Fresnel Microlens Array: {rows}×{cols}, f = {actual_focal_length:.1f} mm\n'
              f'ROI: {roi_width}×{roi_height} pixels, Each microlens: {lens_width}×{lens_height} pixels '
              f'({lens_width*PIXEL_SIZE*1e6:.1f}×{lens_height*PIXEL_SIZE*1e6:.1f} μm)\n'
              f'Diffraction limited spot size: {airy_disk:.2f} μm | '
              f'Steering: θx={angle_x_deg:.2f}°, θy={angle_y_deg:.2f}° | '
              f'2π={two_pi_value}')
    
    # Draw grid lines to show the microlens boundaries within ROI
    if rows <= 20 and cols <= 20:  # Only draw grid for reasonable number of lenses
        for r in range(1, rows):
            y_line = roi_top + r * lens_height - 0.5
            plt.axhline(y=y_line, color='g', linestyle='-', alpha=0.3)
        for c in range(1, cols):
            x_line = roi_left + c * lens_width - 0.5
            plt.axvline(x=x_line, color='g', linestyle='-', alpha=0.3)
    
    plt.legend(loc='upper right')
    plt.show()
    
    return phi

# Function to create binary checkerboard pattern
def create_checkerboard(shape):
    """
    Create a binary checkerboard pattern (alternating 0 and π phase)
    
    Parameters:
    -----------
    shape : tuple
        The shape of the SLM (height, width)
    
    Returns:
    --------
    pattern : ndarray
        The checkerboard pattern (0 and π)
    """
    y, x = np.indices(shape)
    return np.pi * ((x + y) % 2)

def increase_focal_length(b):
    # Increase focal length by 1mm (within limits)
    new_value = min(focal_length_coarse_widget.max, focal_length_coarse_widget.value + 1)
    focal_length_coarse_widget.value = new_value

def decrease_focal_length(b):
    # Decrease focal length by 1mm (within limits)
    new_value = max(focal_length_coarse_widget.min, focal_length_coarse_widget.value - 1)
    focal_length_coarse_widget.value = new_value

focal_increase_button = widgets.Button(
    description='+1mm',
    tooltip='Increase focal length by 1mm',
    button_style='success',
    icon='plus'
)
focal_increase_button.on_click(increase_focal_length)

focal_decrease_button = widgets.Button(
    description='-1mm',
    tooltip='Decrease focal length by 1mm',
    button_style='danger',
    icon='minus'
)
focal_decrease_button.on_click(decrease_focal_length)

# Function to reset all parameters to default values
def reset_parameters(button):
    focal_length_coarse_widget.value = DEFAULT_FL_MM
    focal_length_fine_widget.value = 0
    lens_type_widget.value = False
    rows_widget.value = 2
    cols_widget.value = 2
    roi_center_x_widget.value = width // 2
    roi_center_y_widget.value = height // 2
    roi_width_widget.value = height
    roi_height_widget.value = height
    angle_x_widget.value = 0
    angle_y_widget.value = 0
    two_pi_value_widget.value = DEFAULT_2PI_VALUE

# Get SLM dimensions
height, width = slm.shape

# Create focal length widgets - separate coarse and fine adjustment
focal_length_coarse_widget = widgets.FloatSlider(
    value=DEFAULT_FL_MM,
    min=210,
    max=260,
    step=1.0,
    description='Focal Length (mm):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

focal_length_fine_widget = widgets.FloatSlider(
    value=0,
    min=-0.9,
    max=0.9,
    step=0.1,
    description='Fine Adjust (mm):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# Create lens type selector
lens_type_widget = widgets.ToggleButtons(
    options=[('Positive Lens', False), ('Negative Lens', True)],
    description='Lens Type:',
    value=False
)

# Create beam steering angle controls
angle_x_widget = widgets.FloatSlider(
    value=0,
    min=-30,
    max=30,
    step=0.1,
    description='Steering θx (mrad):',
    continuous_update=False,
    style={'description_width': 'initial'},
    readout_format='.1f'
)

angle_y_widget = widgets.FloatSlider(
    value=0,
    min=-30,
    max=30,
    step=0.1,
    description='Steering θy (mrad):',
    continuous_update=False,
    style={'description_width': 'initial'},
    readout_format='.1f'
)

# Create 2π calibration value control
two_pi_value_widget = widgets.IntText(
    value=DEFAULT_2PI_VALUE,
    min=1,
    max=190,
    step=1,
    description='2π SLM Value:',
    continuous_update=False,
    style={'description_width': 'initial'},
    tooltip='SLM grayscale value corresponding to 2π phase shift'
)

# Create microlens array controls
rows_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Rows:',
    continuous_update=False
)

cols_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Columns:',
    continuous_update=False
)

# Create ROI controls
roi_center_x_widget = widgets.IntSlider(
    value=width // 2,
    min=0,
    max=width-1,
    step=1,
    description='ROI Center X:',
    continuous_update=False
)

roi_center_y_widget = widgets.IntSlider(
    value=height // 2,
    min=0,
    max=height-1,
    step=1,
    description='ROI Center Y:',
    continuous_update=False
)

roi_width_widget = widgets.IntSlider(
    value=height,
    min=10,
    max=width,
    step=1,
    description='ROI Width:',
    continuous_update=False
)

roi_height_widget = widgets.IntSlider(
    value=height,
    min=10,
    max=height,
    step=1,
    description='ROI Height:',
    continuous_update=False
)

# Add information about microlens dimensions and Airy disk
def update_lens_info(rows, cols, roi_width, roi_height, focal_length, is_negative, 
                     angle_x_mrad, angle_y_mrad, two_pi_value):
    lens_width = roi_width // cols
    lens_height = roi_height // rows
    
    # Calculate Airy disk diameter
    actual_focal_length = -focal_length if is_negative else focal_length
    lens_width_m = lens_width * PIXEL_SIZE
    lens_height_m = lens_height * PIXEL_SIZE
    airy_disk = calculate_airy_disk(abs(actual_focal_length) * 1e-3, lens_width_m, lens_height_m)
    
    # Calculate deflection angles in degrees
    angle_x_deg = angle_x_mrad * 0.0573
    angle_y_deg = angle_y_mrad * 0.0573
    
    # Calculate lateral shift at focal plane
    shift_x = abs(actual_focal_length) * 1e-3 * np.tan(angle_x_mrad * 1e-3) * 1e6  # in μm
    shift_y = abs(actual_focal_length) * 1e-3 * np.tan(angle_y_mrad * 1e-3) * 1e6  # in μm
    
    return (f"<b>Each microlens:</b> {lens_width}×{lens_height} pixels "
            f"({lens_width*PIXEL_SIZE*1e6:.1f}×{lens_height*PIXEL_SIZE*1e6:.1f} μm)<br>"
            f"<b>F-number:</b> {abs(actual_focal_length)*1e-3/((lens_width+lens_height)*PIXEL_SIZE/2):.2f}<br>"
            f"<b>Diffraction limited spot size:</b> {airy_disk:.2f} μm<br>"
            f"<b>Beam steering:</b> θx={angle_x_deg:.2f}°, θy={angle_y_deg:.2f}°<br>"
            f"<b>Focal plane shift:</b> Δx={shift_x:.1f} μm, Δy={shift_y:.1f} μm<br>"
            f"<b>2π calibration:</b> {two_pi_value} (current setting)")

lens_info = widgets.HTML(
    value=update_lens_info(2, 2, width // 2, height // 2, 100, False, 0, 0, DEFAULT_2PI_VALUE)
)

def update_info(change):
    lens_info.value = update_lens_info(
        rows_widget.value, 
        cols_widget.value,
        roi_width_widget.value,
        roi_height_widget.value,
        focal_length_coarse_widget.value + focal_length_fine_widget.value,
        lens_type_widget.value,
        angle_x_widget.value,
        angle_y_widget.value,
        two_pi_value_widget.value
    )

# Add observers for all parameters that affect the lens info
focal_length_coarse_widget.observe(update_info, names='value')
focal_length_fine_widget.observe(update_info, names='value')
rows_widget.observe(update_info, names='value')
cols_widget.observe(update_info, names='value')
roi_width_widget.observe(update_info, names='value')
roi_height_widget.observe(update_info, names='value')
lens_type_widget.observe(update_info, names='value')
angle_x_widget.observe(update_info, names='value')
angle_y_widget.observe(update_info, names='value')
two_pi_value_widget.observe(update_info, names='value')

# Reset button
reset_button = widgets.Button(
    description='Reset Parameters',
    button_style='warning',
    tooltip='Reset all parameters to default values',
    icon='refresh'
)
reset_button.on_click(reset_parameters)

# Update parameters function
def update_with_parameters(
    focal_length_coarse, focal_length_fine, rows, cols, 
    roi_center_x, roi_center_y, roi_width, roi_height, is_negative,
    angle_x_mrad, angle_y_mrad, two_pi_value
):
    return update_phase(
        focal_length_coarse, focal_length_fine, rows, cols, 
        roi_center_x, roi_center_y, roi_width, roi_height, is_negative,
        angle_x_mrad, angle_y_mrad, two_pi_value
    )

# Create an interactive output
out = widgets.interactive_output(
    update_with_parameters,
    {
        'focal_length_coarse': focal_length_coarse_widget,
        'focal_length_fine': focal_length_fine_widget,
        'rows': rows_widget,
        'cols': cols_widget,
        'roi_center_x': roi_center_x_widget,
        'roi_center_y': roi_center_y_widget,
        'roi_width': roi_width_widget,
        'roi_height': roi_height_widget,
        'is_negative': lens_type_widget,
        'angle_x_mrad': angle_x_widget,
        'angle_y_mrad': angle_y_widget,
        'two_pi_value': two_pi_value_widget
    }
)

# Display the widgets and output
controls = widgets.VBox([
    lens_type_widget,
    widgets.HBox([focal_length_coarse_widget, focal_length_fine_widget]),
    widgets.HBox([focal_decrease_button, focal_increase_button], layout=widgets.Layout(justify_content='center')),
    widgets.Label("Beam Steering Control:"),
    widgets.HBox([angle_x_widget, angle_y_widget]),
    widgets.Label("Microlens Array Configuration:"),
    widgets.HBox([rows_widget, cols_widget]),
    widgets.Label("ROI Configuration:"),
    widgets.HBox([roi_center_x_widget, roi_center_y_widget]),
    widgets.HBox([roi_width_widget, roi_height_widget]),
    widgets.Label("SLM Calibration:"),
    two_pi_value_widget,
    lens_info,
    reset_button
])

# Display the UI
display(widgets.VBox([controls, out]))

In [15]:
import numpy as np

def generate_and_load_fresnel_lens(slm, focal_length_mm, angle_x_mrad, angle_y_mrad, 
                                   wavelength=550e-9, pixel_size=9.2e-6, two_pi_value=190):
    """
    Generate and load a Fresnel lens phase pattern with beam steering to SLM.
    
    Parameters:
    -----------
    slm : object
        The SLM object with .shape attribute and .set_phase() method
    focal_length_mm : float
        Focal length in millimeters (positive for converging, negative for diverging)
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    wavelength : float
        Wavelength of light in meters (default: 550nm)
    pixel_size : float
        Size of the SLM pixel in meters (default: 9.2μm)
    two_pi_value : int
        SLM grayscale value corresponding to 2π phase shift (default: 190)
    
    Returns:
    --------
    phase_uint8 : ndarray
        The phase pattern as uint8 array that was loaded to the SLM
    """
    # Get SLM shape
    height, width = slm.shape
    
    # Convert focal length to meters
    focal_length_m = focal_length_mm * 1e-3
    
    # Create coordinate grid
    y, x = np.indices((height, width))
    
    # Calculate center of the SLM
    center_x = width / 2
    center_y = height / 2
    
    # Calculate distances from center in meters
    x_meters = (x - center_x) * pixel_size
    y_meters = (y - center_y) * pixel_size
    r_squared = x_meters**2 + y_meters**2
    
    # Calculate Fresnel lens phase
    # φ(x,y) = (2π/λ) * (f - sqrt(f² + x² + y²))
    f_squared = focal_length_m**2
    lens_phase = (2 * np.pi / wavelength) * (focal_length_m - np.sqrt(f_squared + r_squared))
    
    # Calculate linear phase for beam steering
    # Convert milliradians to radians
    angle_x_rad = angle_x_mrad * 1e-3
    angle_y_rad = angle_y_mrad * 1e-3
    
    # Phase gradients for steering
    phase_gradient_x = 2 * np.pi * np.sin(angle_x_rad) / wavelength
    phase_gradient_y = 2 * np.pi * np.sin(angle_y_rad) / wavelength
    
    # Linear phase for beam steering
    steering_phase = phase_gradient_x * x_meters + phase_gradient_y * y_meters
    
    # Combine lens and steering phases
    total_phase = lens_phase + steering_phase
    
    # Wrap phase to [0, 2π) range
    total_phase = total_phase % (2 * np.pi)
    
    # Convert to uint8 for SLM
    phase_uint8 = np.uint8(total_phase / (2 * np.pi) * two_pi_value)
    
    # Load to SLM
    slm.set_phase(phase_uint8)
    
    return phase_uint8


# Example usage:
phase = generate_and_load_fresnel_lens(slm, 
                                       focal_length_mm=238, 
                                       angle_x_mrad=0, 
                                       angle_y_mrad=0)